In [28]:
from typing import Dict
from tqdm.auto import tqdm

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import DBSCAN

import numpy as np
import random
import pandas as pd
import ydata_profiling

import scienceplots
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

import torch
import os

%matplotlib inline

pd.plotting.register_matplotlib_converters()
sns.set_style("whitegrid")
sns.set_palette("RdBu")
sns.set(
    rc={'text.usetex': True},
    font="serif",
    font_scale=1.2
)


# 特征建立

In [29]:
days = {i: pd.read_pickle(f'../Datasets/day{str(i).zfill(2)}.pkl')
        for i in range(1, 22)}


In [30]:
dfs = []
for i in range(1, 22):
    pt = pd.DataFrame(pd.DataFrame(days[i].pivot_table(
        index='uid',
        values='duration',
        columns='app_class',
        aggfunc=np.sum
    )['a']).to_records())
    pt['day'] = i
    dfs.append(pt)

features = pd.DataFrame(pd.concat(dfs, ignore_index=True).pivot_table(
    index='uid',
    values='a',
    columns='day',
).fillna(0).to_records())


In [31]:
# features_q2 = features.merge(
#     features_user2cluster, on='uid', how='left').fillna(404)

# features_q2['cluster'] = features_q2['cluster'].astype('int')
# features_q2['cluster'] = features_q2['cluster'].astype('str')
# features_q2['cluster'] = features_q2['cluster'].replace({
#     '0': 'AA', '1': 'BB', '2': 'CC', '3': 'DD', '4': 'EE', '404': 'NaN'
# })


In [35]:
features.to_pickle('../Datasets/features_q2.pkl')


In [36]:
features


,uid,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0000348F61881026FE0036840F25309F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0000B0540F08BB99C22D22219A1B47A2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1419.0,...,0.0,0.0,1421.0,0.0,0.0,1387.0,2034.0,0.0,393.0,0.0
2,00022474E06735ACCAC428E1AC7EAC16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2143.0,81.0,...,303.0,609.0,66.0,1246.0,159.0,899.0,1367.0,0.0,0.0,0.0
3,00028CD7D203AF88C4FE4D0A83A0016E,50.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,315.0
4,000343EFFAAF27FE66CAD41434814C71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47340,FFF62C1EEF706DF3174EF676391E15C1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47341,FFF6689AA115DFBAA4A6D66928F93971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47342,FFF82C898EE523DA1F4137097FECCB20,0.0,0.0,676.0,12.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,68.0,0.0,0.0,0.0
47343,FFFA25131B53BDF8B855AB4D473E29C2,278.0,206.0,5981.0,1061.0,865.0,479.0,135.0,1.0,227.0,...,300.0,1015.0,1846.0,965.0,828.0,0.0,1615.0,1924.0,2090.0,2885.0
